In [1]:
using JuMP 
using Mosek

In [ ]:
#Script para el aprendizaje de grafos usando la metrica de Wasserstein, pero esta vez con la funcion de 
#costo ||.||1 que vuelve al problema lineal.

In [ ]:
# @param:weights : weights of the observed matrices. Sum of weights must be 1. and non-negative. the size is equal to the amount
#of observed graphs.
#@param: delta : size of the ball of the Wasserstein metric. delta > 0.
#@param: A : cluster form of the data. A entries must be 0 or 1. 
#@param: B : vector of matrices containing the observations of the graph B. If X,Y are matrices, an array of matrices
#is of the form Array[X,Y].
#@param: n : the dimension of the matrices.
#@param:N : number of observations.
#The set over which the is problem is optimized is:

In [ ]:
function learnLinearGraph(weights,delta,B,n)
    N = length(B)
    one =ones(n)
    m = Model(solver=MosekSolver())
    @variable(m,lambda>=0)
    @variable(m,S[1:N])
    @variable(m,A[1:n,1:n], Symmetric)
    #W debe ser simetrica? Creo que no.
    @variable(m, W[1:n,1:n],Symmetric)
    #Q es una variable auxiliar, juega el papel de Lambda en la demostracion
    @variable(m,Q[1:n,1:n])
    # u es una variable auxiliar para calcular la norma infinito de W.
    @variable(m,u>=0)
    #Z es un conjunto de matrices que permite calcular la norma 1 de A-B[i].
    Z = [@variable(m, [1:n, 1:n], Symmetric) for p in 1:N]
 
    #Constrains...
    #constrains for lambda already in the definition of the varible.
    #Constrains for A.
    @constraint(m,A.>=0)
    @constraint(m,A.<=1)
    #Constrains for Z.
    [@constraint(m, -Z[p].<= A-B[p]) for p in 1:N]
    [@constraint(m,  A-B[p].<=Z[p]) for p in 1:N]
    #Constrains for W. 
    @constraint(m,u<=lambda)
    @constraint(m,u .>=W)
    @constraint(m,u .>=-W)
     #Constrains for Q.
    @constraint(m,Q .>=-(2*A-one*one'-W))
    @constraint(m,Q .>=0)
    #[@constraint(m,2*A-one*one'-W+O[p]+E[p].==0) for p in 1:N]
    [@constraint(m, ones(n)'*Q*ones(n)<=S[p]-trace((2*A-one*one'-W)'*B[p])) for p in 1:N]
    #Objetivo
    @objective(m,Min,lambda*delta+ dot(weights,S+prod.(Z,n)))
    status = solve(m)
    return([getvalue(A),getvalue(S),getvalue(lambda),getvalue(W),getobjectivevalue(m)])
end

In [57]:
function learnLinearError(weights,delta,A,B,n)
    N = length(B)
    one =ones(n)
    m = Model(solver=MosekSolver())
    @variable(m,lambda>=0)
    @variable(m,S[1:N])
    #W debe ser simetrica? Creo que no.
    @variable(m, W[1:n,1:n],Symmetric)
    #Q es una variable auxiliar, juega el papel de Lambda en la demostracion
    @variable(m,Q[1:n,1:n])
    # u es una variable auxiliar para calcular la norma infinito de W.
    @variable(m,u>=0)
    #Z es un conjunto de matrices que permite calcular la norma 1 de A-B[i].
    Z = [@variable(m, [1:n, 1:n], Symmetric) for p in 1:N]
    #Constrains...
    #constrains for lambda already in the definition of the varible.
    #Constrains for Z.
    [@constraint(m, -Z[p].<= A-B[p]) for p in 1:N]
    [@constraint(m,  A-B[p].<=Z[p]) for p in 1:N]
    #Constrains for W. 
    @constraint(m,u<=lambda)
    @constraint(m,u .>=W)
    @constraint(m,u .>=-W)
    #Constrains for Q.
    @constraint(m,Q .>=-(2*A-one*one'-W))
    @constraint(m,Q .>=0)
    [@constraint(m, ones(n)'*Q*ones(n)<=S[p]-trace((2*A-one*one'-W)'*B[p])) for p in 1:N]
    #Objetivo
    @objective(m,Min,lambda*delta+ dot(weights,S+prod.(Z,n)))
    status = solve(m)
    return([getvalue(Q),getvalue(lambda),getvalue(S),getobjectivevalue(m)])
end

learnLinearError (generic function with 1 method)

In [55]:
#papa1 = Array{Any}(2)
#papa1[1]=erdosgraph([0.8,1,1],0.1,[5,3,4],12)
#papa1[2]=erdosgraph([0.8,1,1],0.1,[5,3,4],12)
#real = erdosgraph([1,1,1],0,[5,3,4],12)
respuesta=learnLinearError([0.5,0.5],0.1,real,papa1,12)

MOSEK warning 710: #2 (nearly) zero elements are specified in sparse col '' (3) of matrix 'A'.
MOSEK warning 710: #2 (nearly) zero elements are specified in sparse col '' (4) of matrix 'A'.
MOSEK warning 710: #1 (nearly) zero elements are specified in sparse col '' (5) of matrix 'A'.
MOSEK warning 710: #1 (nearly) zero elements are specified in sparse col '' (8) of matrix 'A'.
MOSEK warning 710: #2 (nearly) zero elements are specified in sparse col '' (9) of matrix 'A'.
MOSEK warning 710: #2 (nearly) zero elements are specified in sparse col '' (10) of matrix 'A'.
MOSEK warning 710: #2 (nearly) zero elements are specified in sparse col '' (11) of matrix 'A'.
MOSEK warning 710: #2 (nearly) zero elements are specified in sparse col '' (12) of matrix 'A'.
MOSEK warning 710: #2 (nearly) zero elements are specified in sparse col '' (13) of matrix 'A'.
MOSEK warning 710: #1 (nearly) zero elements are specified in sparse col '' (14) of matrix 'A'.
Warning number 710 is disabled.
Problem
  Nam

3-element Array{Any,1}:
  [-0.0 -0.0 … -0.0 -0.0; -0.0 -0.0 … -0.0 -0.0; … ; -0.0 -0.0 … -0.0 9.32587e-14; -0.0 -0.0 … 9.32587e-14 -0.0]
 1.0                                                                                                            
  [-1.2669e-11, -1.26761e-11]                                                                                   